In [1]:
import pandas as pd
import re

file = 'DB_Word_Mid.csv'
data = pd.read_csv(file)
df = data.copy()
df.head(5)

,textbook ID,학년,출판사(저자),교육과정,레슨,단어,품사,PoS,뜻,예문,예문 해석,영영풀이
0,NaN,중1,동아(이),2022,SL,12th (twelfth),"명사, 형용사",NaN,"(형) 열두 번째의; (명) 열두 번째, 12일",The lawyer's office is on the twelfth floor.,그 변호사 사무실은 12층에 있다.,"12 in order, following 11th but before 13th"
1,NaN,중1,능률(양),2015,1,a,관사,10,"하나의, 한 사람의",A baby is crying.,한 아기가 울고 있다.,NaN
2,NaN,중3,비상,2015,3,a bit,부사,4,"(부) 조금, 약간",He said the movie was a bit boring.,그는 그 영화가 약간 지루하다고 말했다.,NaN
3,NaN,중3,YBM(박),2015,6,a couple of,형용사,3,"두서너 개의, 몇 개의",A couple of police officers were standing at t...,두서너 명의 경찰관이 문 앞에 서 있었다.,"approximately two or three, few"
4,NaN,중3,YBM(송),2015,2,a couple of,형용사,3,"두서너 개의, 몇 개의, 몇 사람의",A couple of boys are playing catch ball.,두서너 명의 남자아이들이 캐치볼을 하고 있다.,two people or things of the same category


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8998 entries, 0 to 8997
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   textbook ID  0 non-null      float64
 1   학년           8998 non-null   object 
 2   출판사(저자)      8998 non-null   object 
 3   교육과정         8998 non-null   int64  
 4   레슨           8998 non-null   object 
 5   단어           8998 non-null   object 
 6   품사           8995 non-null   object 
 7   PoS          8712 non-null   object 
 8   뜻            8998 non-null   object 
 9   예문           8368 non-null   object 
 10  예문 해석        8368 non-null   object 
 11  영영풀이         4195 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 843.7+ KB


In [3]:
df.columns

Index(['textbook ID', '학년', '출판사(저자)', '교육과정', '레슨', '단어', '품사', 'PoS', '뜻',
       '예문', '예문 해석', '영영풀이'],
      dtype='object')

## TEXT ID

In [5]:
id = pd.read_csv('../DB/Text_ID.csv')
id.columns
drop_col = ['구분','교과서 확보', '어휘 자료','function', 'structure', 'Issue']
id.drop(columns=drop_col, inplace=True)
id['학년 정보'] = id['학년 정보'].astype(str).replace('영어', '', regex=True)
id.rename(columns={
    '학년 정보': '학년',
    '개정 정보': '교육과정'
}, inplace=True)
display(id)

,textbook ID,학년,교육과정,출판사
0,301,초3,2015,천재 (함순애)
1,302,초3,2015,YBM (김혜리)
2,303,초3,2015,YBM (최은경)
3,304,초3,2015,동아 (박기화)
4,305,초3,2015,아이스크림
...,...,...,...,...
88,905,중3,2015,비상 (김진완)
89,906,중3,2015,천재 (이재영)
90,907,중3,2015,천재 (정사열)
91,908,중3,2015,YBM (박준언)


In [6]:
# 변환 함수 정의
def modify_publisher(name):
    if "(" in name and ")" in name:  # 저자 이름이 있는 경우만 처리
        parts = name.split("(")  # 괄호를 기준으로 분리
        publisher = parts[0].strip()  # 출판사 이름
        author_initial = parts[1][0]  # 저자의 첫 글자만 추출
        return f"{publisher}({author_initial})"
    return name  # 저자가 없는 경우 그대로 반환

# 데이터 변환 적용
id["출판사"] = id["출판사"].apply(modify_publisher)

In [7]:
id['출판사'] = id['출판사'].replace('비상(김)','비상')

In [10]:
df['출판사(저자)'].value_counts()

출판사(저자)
능률(양)     1384
YBM(박)    1307
YBM(송)    1293
비상        1194
동아(윤)      867
동아(이)      855
천재(정)      720
천재(이)      606
능률(김)      601
YBM(김)     171
Name: count, dtype: int64

In [11]:
id['출판사'].value_counts()

출판사
천재(함)      8
천재(이)      8
동아(윤)      8
YBM(김)     7
YBM(최)     6
아이스크림      6
미래엔        6
능률(김)      4
YBM(박)     4
동아(이)      4
천재(김)      4
동아(박)      4
능률(양)      3
비상         3
천재(정)      3
YBM(송)     3
시사(최)      2
대교(이)      2
금성(최)      1
다락원(강)     1
미래(최)      1
지학사(민)     1
천재(소)      1
미래엔(문)     1
비상교육(황)    1
지학사(송)     1
Name: count, dtype: int64

In [12]:
id.head(-5)

,textbook ID,학년,교육과정,출판사
0,301,초3,2015,천재(함)
1,302,초3,2015,YBM(김)
2,303,초3,2015,YBM(최)
3,304,초3,2015,동아(박)
4,305,초3,2015,아이스크림
...,...,...,...,...
83,809,중2,2015,YBM(송)
84,901,중3,2015,능률(김)
85,902,중3,2015,능률(양)
86,903,중3,2015,동아(윤)


In [13]:
id['학년'].value_counts()

학년
중1     23
초3     14
초4     14
초5     12
초6     12
중2      9
중3      9
Name: count, dtype: int64

In [14]:
id.head(-1)

,textbook ID,학년,교육과정,출판사
0,301,초3,2015,천재(함)
1,302,초3,2015,YBM(김)
2,303,초3,2015,YBM(최)
3,304,초3,2015,동아(박)
4,305,초3,2015,아이스크림
...,...,...,...,...
87,904,중3,2015,동아(이)
88,905,중3,2015,비상
89,906,중3,2015,천재(이)
90,907,중3,2015,천재(정)


In [15]:
id['학년'] = id['학년'].str.strip()  # 공백 제거

In [16]:
id.loc[id['학년'] == '중1']

,textbook ID,학년,교육과정,출판사
52,701,중1,2015,능률(김)
53,702,중1,2015,능률(양)
54,703,중1,2015,동아(윤)
55,704,중1,2015,동아(이)
56,705,중1,2015,비상
57,706,중1,2015,천재(이)
58,707,중1,2015,천재(정)
59,708,중1,2015,YBM(박)
60,709,중1,2015,YBM(송)
61,710,중1,2015,금성(최)


In [20]:
filtered_data = id.loc[id['학년'].isin(['중1', '중2', '중3'])]
filtered_data = filtered_data.rename(columns={'출판사' : "출판사(저자)"})
filtered_data.head()

,textbook ID,학년,교육과정,출판사(저자)
52,701,중1,2015,능률(김)
53,702,중1,2015,능률(양)
54,703,중1,2015,동아(윤)
55,704,중1,2015,동아(이)
56,705,중1,2015,비상


In [23]:
df.drop(columns="textbook ID", axis=1, inplace=True)
df.head()

,학년,출판사(저자),교육과정,레슨,단어,품사,PoS,뜻,예문,예문 해석,영영풀이
0,중1,동아(이),2022,SL,12th (twelfth),"명사, 형용사",NaN,"(형) 열두 번째의; (명) 열두 번째, 12일",The lawyer's office is on the twelfth floor.,그 변호사 사무실은 12층에 있다.,"12 in order, following 11th but before 13th"
1,중1,능률(양),2015,1,a,관사,10,"하나의, 한 사람의",A baby is crying.,한 아기가 울고 있다.,NaN
2,중3,비상,2015,3,a bit,부사,4,"(부) 조금, 약간",He said the movie was a bit boring.,그는 그 영화가 약간 지루하다고 말했다.,NaN
3,중3,YBM(박),2015,6,a couple of,형용사,3,"두서너 개의, 몇 개의",A couple of police officers were standing at t...,두서너 명의 경찰관이 문 앞에 서 있었다.,"approximately two or three, few"
4,중3,YBM(송),2015,2,a couple of,형용사,3,"두서너 개의, 몇 개의, 몇 사람의",A couple of boys are playing catch ball.,두서너 명의 남자아이들이 캐치볼을 하고 있다.,two people or things of the same category


## 데이터 합치기

In [24]:
merged_data = df.merge(filtered_data, on=["학년", "출판사(저자)", "교육과정"], how="left")
merged_data = merged_data[["textbook ID"] + [col for col in merged_data.columns if col != "textbook ID"]]
merged_data['textbook ID'] = merged_data['textbook ID'].astype(int)

In [25]:
merged_data.head()

,textbook ID,학년,출판사(저자),교육과정,레슨,단어,품사,PoS,뜻,예문,예문 해석,영영풀이
0,716,중1,동아(이),2022,SL,12th (twelfth),"명사, 형용사",NaN,"(형) 열두 번째의; (명) 열두 번째, 12일",The lawyer's office is on the twelfth floor.,그 변호사 사무실은 12층에 있다.,"12 in order, following 11th but before 13th"
1,702,중1,능률(양),2015,1,a,관사,10,"하나의, 한 사람의",A baby is crying.,한 아기가 울고 있다.,NaN
2,905,중3,비상,2015,3,a bit,부사,4,"(부) 조금, 약간",He said the movie was a bit boring.,그는 그 영화가 약간 지루하다고 말했다.,NaN
3,908,중3,YBM(박),2015,6,a couple of,형용사,3,"두서너 개의, 몇 개의",A couple of police officers were standing at t...,두서너 명의 경찰관이 문 앞에 서 있었다.,"approximately two or three, few"
4,909,중3,YBM(송),2015,2,a couple of,형용사,3,"두서너 개의, 몇 개의, 몇 사람의",A couple of boys are playing catch ball.,두서너 명의 남자아이들이 캐치볼을 하고 있다.,two people or things of the same category


In [26]:
file_name = 'Word_Mid.csv'
merged_data.to_csv(file_name, encoding='utf-8-sig')

In [28]:
test = pd.read_csv('Word_Mid.csv')
test.head()

,Unnamed: 0,textbook ID,학년,출판사(저자),교육과정,레슨,단어,품사,PoS,뜻,예문,예문 해석,영영풀이
0,0,716,중1,동아(이),2022,SL,12th (twelfth),"명사, 형용사",NaN,"(형) 열두 번째의; (명) 열두 번째, 12일",The lawyer's office is on the twelfth floor.,그 변호사 사무실은 12층에 있다.,"12 in order, following 11th but before 13th"
1,1,702,중1,능률(양),2015,1,a,관사,10,"하나의, 한 사람의",A baby is crying.,한 아기가 울고 있다.,NaN
2,2,905,중3,비상,2015,3,a bit,부사,4,"(부) 조금, 약간",He said the movie was a bit boring.,그는 그 영화가 약간 지루하다고 말했다.,NaN
3,3,908,중3,YBM(박),2015,6,a couple of,형용사,3,"두서너 개의, 몇 개의",A couple of police officers were standing at t...,두서너 명의 경찰관이 문 앞에 서 있었다.,"approximately two or three, few"
4,4,909,중3,YBM(송),2015,2,a couple of,형용사,3,"두서너 개의, 몇 개의, 몇 사람의",A couple of boys are playing catch ball.,두서너 명의 남자아이들이 캐치볼을 하고 있다.,two people or things of the same category
